# Pipeline

### Steps

1. Import packages and define folders
2. Choose which files to use as input
3. Instantiate functions
4. Run Program

In [1]:
# Importing other necessary packages
import glob
import pandas as pd
import os
from pathlib import Path
import shutil

# Import helper functions tying everything together
from scanner_functions import move_files
from scanner_functions import ocr_pipeline

## setting folders and pdfs
folder = "/home/hennes/Internship/pdfs/" # should be folder containing pdfs of election
save_folder = '/home/hennes/Internship/constituencies/' # folder into which csvs should be saved
saved = [os.path.splitext(csv)[0] for csv in next(os.walk(save_folder))[2]]
old = '/home/hennes/Internship/old_files/' # folder into which old files are moved
old_files = [folder for folder in next(os.walk(old))[1]]
allpdf = [pdf for pdf in glob.glob(folder+'*') if pdf.endswith(".pdf")] # list with all pdfs from folder

## Choosing files (Choose one of the following options)

### input = pdfs

all pdfs that do not have a corresponding file in output folder

In [2]:
# exclude pdfs for which there is already a csv in save folder
pdflist = sorted([pdf for pdf in allpdf if pdf.split('/')[-1].split('_')[0].split('.')[0] not in
           [file.split('/')[-1].split('.')[0] for file in glob.glob(save_folder+'*')]])

pdfs as per excel sheet

In [9]:
df = pd.read_excel('/home/hennes/Downloads/Book.xlsx')

# get relevant pdf numbers

worklist = df[df['Comments'] == 'redo with threshold']['Constituency number'].tolist()

# give appropriate filename endings to items

for idx, item in enumerate(worklist):
    if len(str(item)) == 1:
        worklist[idx] = f'AC00{item}.pdf'
    if len(str(item)) == 2:
        worklist[idx] = f'AC0{item}.pdf'
    if len(str(item)) == 3:
        worklist[idx] = f'AC{item}.pdf'
        
worklist = tuple(worklist)
pdflist = [pdf for pdf in allpdf if pdf.endswith(worklist)]

### input = files

all ACs that do not have a corresponding file in output folder

In [27]:
# creates list of lists with each list containing the folders of one AC constituency
filelist = []
for e in set(sorted([folder.split('-')[1] for folder in next(os.walk(old))[1] if folder.split('-')[1] not in saved])):
    filelist.append(sorted([folder for folder in old_files if folder.split('-')[1] in e]))
filelist = sorted(filelist)

All ACs as per excel sheet

In [9]:
filelist = []
for e in set(sorted([folder.split('-')[1] for folder in next(os.walk(old))[1]])):
    filelist.append(sorted([folder for folder in old_files if folder.split('-')[1] in e]))
filelist = sorted(filelist)

In [2]:
# first get list of lists with all files

filelist = []
for e in set(sorted([folder.split('-')[1] for folder in next(os.walk(old))[1]])):
    filelist.append(sorted([folder for folder in old_files if folder.split('-')[1] in e]))
filelist = sorted(filelist)


# then filter as per excel
df = pd.read_excel('/home/hennes/Downloads/Book.xlsx')

# get relevant pdf numbers

worklist = df[df['Comments'] == 'redo with threshold']['Constituency number'].tolist()

# give appropriate filename endings to items

for idx, item in enumerate(worklist):
    if len(str(item)) == 1:
        worklist[idx] = f'AC00{item}'
    if len(str(item)) == 2:
        worklist[idx] = f'AC0{item}'
    if len(str(item)) == 3:
        worklist[idx] = f'AC{item}'
        
worklist = tuple(worklist)

filelist = [l for l in filelist if l[0].split('-')[1] in worklist]

## Running Program

Cleaning eventual files still in pdf folder.

In [4]:
move_files(folder, old)

For when input is folders from old folder (tables already extracted):

In [5]:
for l in filelist:
    [shutil.copytree(old+x, folder+x) for x in l]
    pdf = f'/{l[0].split("-")[1]}'
    print(f' \nWORKING ON {pdf.split("/")[-1]}\n ')
    stop = ocr_pipeline(pdf, from_cell=True, thresh = True, no_noise = True)
    if stop:
        break

 
WORKING ON AC068
 
extracted cells of AC068
completed ocr of AC068
working on PC010-AC068-11
working on PC010-AC068-12
working on PC010-AC068-13
working on PC010-AC068-14
working on PC010-AC068-15
working on PC010-AC068-16
working on PC010-AC068-17
working on PC010-AC068-18
working on PC010-AC068-19
working on PC010-AC068-20
working on PC010-AC068-21
Saved AC068 to folder.
 
WORKING ON AC069
 
extracted cells of AC069
completed ocr of AC069
working on PC010-AC069-22
working on PC010-AC069-23
working on PC010-AC069-24
working on PC010-AC069-25
working on PC010-AC069-26
working on PC010-AC069-27
working on PC010-AC069-28
Saved AC069 to folder.
 
WORKING ON AC070
 
extracted cells of AC070
completed ocr of AC070
working on PC010-AC070-29
working on PC010-AC070-30
working on PC010-AC070-31
working on PC010-AC070-32
working on PC010-AC070-33
working on PC010-AC070-34
working on PC010-AC070-35
working on PC010-AC070-36
working on PC010-AC070-37
working on PC010-AC070-38
working on PC010-AC

For when input is image files from old folder:

In [4]:
pdflist = [e.split('.')[0] for e in worklist]

NameError: name 'worklist' is not defined

In [3]:
pdflist = allpdf

In [4]:
pdflist

['/home/hennes/Internship/pdfs/PC007.pdf']

In [4]:
for x in pdflist:
    imglist = [img for img in next(os.walk(old))[2] if img.startswith(os.path.split(x)[1].split('.')[0])]
    [shutil.copy(old+x, folder) for x in imglist]
    pdf = f'{folder}/{x}.pdf'
    print(f' \nWORKING ON {x}\n ')
    stop = ocr_pipeline(pdf, no_noise = True)
    if stop:
        break

NameError: name 'shutil' is not defined

For when input files are pdf:

In [20]:
# no options
pdflist = allpdf

In [21]:
for x in pdflist:
    print(f' \nWORKING ON {x}\n ')
    stop = ocr_pipeline(x, AC = True)
    if stop:
        break

 
WORKING ON /home/hennes/Internship/pdfs/PC010.pdf
 
created images of PC010.pdf
preprocessed images of PC010.pdf
PC010-01: AC = 67
PC010-02: AC = 67
PC010-03: AC = 67
PC010-04: AC = 67
PC010-05: AC = 67
PC010-06: AC = 67
PC010-07: AC = 67
PC010-08: AC = 67
PC010-09: AC = 67
PC010-10: AC = 67
PC010-11: AC = 68
PC010-12: AC = 68
PC010-13: AC = 68
PC010-14: AC = 68
PC010-15: AC = 68
PC010-16: AC = 68
PC010-17: AC = 68
PC010-18: AC = 68
PC010-19: AC = 68
PC010-20: AC = 68
PC010-21: AC = 68
PC010-22: AC = 69
PC010-23: AC = 69
PC010-24: AC = 69
PC010-25: AC = 69
PC010-26: AC = 69
PC010-27: AC = 69
PC010-28: AC = 69
PC010-29: AC = 70
PC010-30: AC = 70
PC010-31: AC = 70
PC010-32: AC = 70
PC010-33: AC = 70
PC010-34: AC = 70
PC010-35: AC = 70
PC010-36: AC = 70
PC010-37: AC = 70
PC010-38: AC = 70
PC010-39: AC = 70
PC010-40: AC = 70
PC010-41: AC = 71
PC010-42: AC = 71
PC010-43: AC = 71
PC010-44: AC = 71
PC010-45: AC = 71
PC010-46: AC = 71
PC010-47: AC = 71
PC010-48: AC = 71
PC010-49: AC = 71
PC0